In [1]:
from robobrowser import RoboBrowser
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html

import datetime
today = datetime.date.today().strftime('%Y-%m-%d')

In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# Coursera
### Collect All Course Links

In [ ]:
base = 'https://www.coursera.org/browse/'

categories = ['arts-and-humanities', 'business', 'computer-science', 'data-science', 'information-technology', 
                'life-sciences', 'math-and-logic', 'personal-development', 'physical-science-and-engineering', 
                'social-sciences', 'language-learning']

In [ ]:
all_courses = []

for category in categories:
    result = 'Something'
    while result != None:
        try:
            page = requests.get(base + category)
            soup = BeautifulSoup((page).content, 'html.parser')
            result = soup.find('h3')
        except: 
            pass
    subjects = [x.get('href').replace('#', '/') for x in soup.findAll("a", {"class": "index-item-link"})]
    
    for subject in subjects:

        for x in range(1, 13):
            url = (base + category + subject + '?page=' + str(x))
            result = 'Something'
            while result != None:
                try:
                    page_2 = requests.get(url)
                    soup_2 = BeautifulSoup((page_2).content, 'html.parser')
                    result = soup_2.find('h3')
                except: 
                    pass
            courses = [x.get('href') for x in soup_2.findAll('a', {'class': 'rc-OfferingCard nostyle'})]
            #print(subject, len(courses), url)
            all_courses.append(courses)

In [ ]:
result = 'Something'
while result != None:
    try:
        page = requests.get('https://www.coursera.org/browse/personal-development')
        soup = BeautifulSoup((page).content, 'html.parser')
        result = soup.find('h3')
    except: 
        pass
courses = [x.get('href') for x in soup.findAll('a', {'class': 'rc-OfferingCard nostyle'})]
all_courses.append(courses)

In [ ]:
result = 'Something'
while result != None:
    try:
        page = requests.get('https://www.coursera.org/browse/math-and-logic')
        soup = BeautifulSoup((page).content, 'html.parser')
        result = soup.find('h3')
    except: 
        pass
courses = [x.get('href') for x in soup.findAll('a', {'class': 'rc-OfferingCard nostyle'})]
all_courses.append(courses)

In [ ]:
#all_courses = list(set([item for sublist in all_courses for item in sublist]))
#pd.DataFrame({'course_links': all_courses}).to_csv('data/jobs/coursera/coursra_course_links.csv', index=False)

### Web Scrape all Courses

In [3]:
all_courses = pd.read_csv('data/jobs/coursera/coursera_course_links.csv')['course_liks'].tolist()

In [4]:
learns = [x for x in all_courses if '/learn/' in x]
learn_groups = list(chunks(learns, 50))
len(learns)

2158

In [5]:
page = requests.get('https://www.coursera.org/learn/songwriting-lyrics')
soup = BeautifulSoup((page).content, 'html.parser')
result = soup.find("div", {'class': 'XdpApp'})

In [14]:
for idx, group in enumerate(learn_groups):
    
    #index = idx + 7
    print(str(idx))
    
    df_list = []
    
    for course in group:
    
        #print(course)
        url = ('https://www.coursera.org' + course)

        result = 'Something'
        tries = 0
        while result != None and tries < 5:
            try:
                page = requests.get(url)
                soup = BeautifulSoup((page).content, 'html.parser')
                result = soup.find("div", {'class': 'XdpApp'})
                tries += 1
            except: 
                pass

        if soup.findAll('h1')[-1]:
            titles = soup.findAll('h1')[-1].text
        else:
            titles = ''
        if soup.find('p', {'class': 'body-1-text course-description'}):
            description = soup.find('p', {'class': 'body-1-text course-description'}).text
        else:
            description = ''
        if soup.find("div", {"class": "ratings-text headline-2-text"}):
            ratings = soup.find("div", {"class": "ratings-text headline-2-text"}).text
        else:
            ratings = ''
        if soup.find("button", {'class': "cdp-view-all-button"}):
            reviews = soup.find("button", {'class': "cdp-view-all-button"}).text
        else:
            reviews = ''
        if soup.find("div", {"class": "headline-1-text creator-names"}):
            institutions = soup.find("div", {"class": "headline-1-text creator-names"}).text
        else:
            institutions = ''
        if soup.findAll("p", {"class": "instructor-name"}):
            instructors = [x.text for x in soup.findAll("p", {"class": "instructor-name"})]
        else:
            instructors = ''
        if soup.find('div', {'class': 'rc-BannerBreadcrumbs caption-text'}):
            subjects = [x.find('a').text for x in soup.find('div', {'class': 'rc-BannerBreadcrumbs caption-text'})]
        else:
            subjects = ''
            

        i = [x.text for x in soup.findAll("span", {"class": "td-title"})]
        j = [x.text for x in soup.findAll("td", {"class": "td-data"})]
        box_cols = pd.DataFrame([j], columns=i)
        
        
        coursera_df = pd.DataFrame({'title': titles,
                       'description': description,
                       'ratings': ratings,
                       'reviews': reviews,
                       'institutions': institutions,
                       'instructors': [instructors],
                       'subjects': [subjects]})
    
        coursera_df = pd.concat([coursera_df, box_cols], axis=1)
        coursera_df['scrape_date'] = today
        coursera_df['url'] = url
    
        df_list.append(coursera_df)
        
    pd.concat(df_list).to_csv('data/jobs/coursera/coursera_data/' + str(idx) + '.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


In [18]:
import glob
files = [pd.read_csv(file) for file in glob.glob("data/jobs/coursera/coursera_data/*.csv")]
coursera = pd.concat(files)
coursera.reset_index(inplace=True, drop=True)

coursera['description'] = coursera['description'].str.replace('About this course: ', '')
coursera['reviews'] = pd.to_numeric(coursera['reviews'].str.lstrip('See all ').str.rstrip(' reviews').str.replace(',', ''))
coursera['rated'] = coursera['ratings'].str.split(' out of 5 of ', expand=True)[0]
coursera['ratings'] = coursera['ratings'].str.split(' out of 5 of ', expand=True)[1]
coursera['rated'] = pd.to_numeric(coursera['rated'].str.lstrip('Rated '))
coursera['ratings'] = pd.to_numeric(coursera['ratings'].str.rstrip(' ratings').str.replace(',', ''))

coursera = coursera.sort_values('ratings', ascending=False)

coursera = coursera[['title', 'url', 'ratings', 'rated', 'reviews', 'description', 'subjects', 'institutions', 'instructors', 
                     'Basic Info', 'Commitment', 'Hardware Req', 'Language', 'Level', 'scrape_date']]

#coursera.to_csv('data/jobs/coursera/coursera_2.csv', index=False)

coursera.head(2)

,title,url,ratings,rated,reviews,description,subjects,institutions,instructors,Basic Info,Commitment,Hardware Req,Language,Level,scrape_date
1514,Machine Learning,https://www.coursera.org/learn/machine-learning,74001.0,4.9,18573.0,Machine learning is the science of getting com...,"['Home', 'Data Science', 'Machine Learning']",Created by: Stanford University,"['Taught by:\xa0\xa0Andrew Ng, Co-founder, Cou...",NaN,NaN,NaN,"English, Subtitles: Spanish, Hindi, Japanese, ...",NaN,2018-05-30
779,Programming for Everybody (Getting Started wit...,https://www.coursera.org/learn/python,37451.0,4.8,10118.0,This course aims to teach everyone the basics ...,"['Home', 'Computer Science', 'Software Develop...",Created by: University of Michigan,"['Taught by:\xa0\xa0Charles Severance, Associa...",Course 1 of 5 in the Python for Everybody Spec...,2-4 hours/week,NaN,"English, Subtitles: German, Chinese (Simplified)",NaN,2018-05-30


#### Trending Courses

In [ ]:
# Pulling data from site
page = requests.get("https://www.coursera.org/featured/trending_courses_locale_en_os_web")

# soup thing
soup = BeautifulSoup(page.content, 'html.parser')

# thing thing
links = [x.get('href') for x in list(soup.findAll('a'))]
links = [x for x in links if '/learn/' in x]

names = []
overviews = []
ratings = []
courses = [x.split('/')[-1] for x in links]

for link in links:
    page = requests.get("https://www.coursera.org/" + link)
    soup = BeautifulSoup((page).content, 'html.parser')
    
    name = [x.text for x in soup.findAll("div", {"class": "module-name"})]
    names.append(name)
    
    overview = [x.text for x in soup.findAll("p", {"class": "body-1-text course-description"})]
    overviews.append(overview)
    
    rating = [x.text for x in soup.findAll("div", {"class": "ratings-text headline-2-text"})]
    ratings.append(rating)
    
coursera_df = pd.DataFrame({'course': courses,
                       'name': names,
                       'overview': overviews,
                       'rating': ratings})

coursera_df.head(8)